imports

In [34]:
import pandas as pd

writer = pd.ExcelWriter('./results/data_of_corrects.xlsx', engine='xlsxwriter')

config

In [35]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_datasheet = r'./results/preprocessed.xlsx'

import df

In [36]:
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df = df.loc[df["Correctness"]==True]
df.to_excel(writer, sheet_name="corrects", index=False)

df_BU_R = df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")].drop("ClickData", axis=1)
df_BU_I = df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")].drop("ClickData", axis=1)
df_TD_R = df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")].drop("ClickData", axis=1)
df_TD_I = df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")].drop("ClickData", axis=1)

df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]

get names of algorithm

In [37]:
algo_name_array = [name for name in df["Algorithm"].unique()]

split dataframes 

In [38]:
df_cat_array = []
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])

df_algo_cat_array = []
for algo_name in algo_name_array:
    tmp_df = []
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])
    df_algo_cat_array.append(tmp_df)

create result

In [39]:
cols = ["Algorithm", "Comprehension",
        "rec correct", "rec total Answers", "rec percent", "R_ResponseTimeMean", "R_ResponseTimeDeviation",
        "it correct", "it total Answers", "it percent", "I_ResponseTimeMean", "I_ResponseTimeDeviation"]

result_df = pd.DataFrame(columns=cols)

count_cols = ["Algorithm", "Comprehension", "Programming", "Right", "Total"]
count_df = pd.DataFrame(columns=count_cols)

for algo_df_array in df_algo_cat_array:
    for i in range(2):
        rec =  algo_df_array[i*2+0]
        it =  algo_df_array[i*2+1]
        
        Algorithm = rec.iloc[0]["Algorithm"]
        Comprehension = "Bottom-Up"
        if i == 1:
            Comprehension = "Top-Down"
            
        count_df = count_df.append(pd.DataFrame([[Algorithm, Comprehension, "R", len(rec.loc[rec["Correctness"]==True]) ,len(rec)]], columns=count_cols))
        count_df = count_df.append(pd.DataFrame([[Algorithm, Comprehension, "I", len(it.loc[it["Correctness"]==True]) ,len(it)]], columns=count_cols))
        
        R_ResponseTimeMean = int(rec["ResponseTime"].mean())
        R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
        
        I_ResponseTimeMean = int(it["ResponseTime"].mean())
        I_ResponseTimeDeviation = int(it["ResponseTime"].std())
        
        rec_cor = len(rec.loc[rec["Correctness"]==True])
        rec_total = len(rec)
        rec_percent = rec_cor/rec_total
        
        it_cor = len(it.loc[it["Correctness"]==True])
        it_total = len(it)
        it_percent = it_cor/it_total
        
        data = [
            Algorithm, Comprehension,
            rec_cor, rec_total, rec_percent, R_ResponseTimeMean, R_ResponseTimeDeviation,
            it_cor, it_total, it_percent, I_ResponseTimeMean, I_ResponseTimeDeviation,
        ]
        
        result_df = result_df.append(pd.DataFrame([data], columns=cols))
        



for i in range(2):
    rec =  df_cat_array[i*2+0]
    it =  df_cat_array[i*2+1]
    
    Algorithm = "Total"
    Comprehension = "Top-Down"
    if i == 1:
        Comprehension = "Bottom-Up"
    
    R_ResponseTimeMean = int(rec["ResponseTime"].mean())
    R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
    
    I_ResponseTimeMean = int(it["ResponseTime"].mean())
    I_ResponseTimeDeviation = int(it["ResponseTime"].std())
    
    rec_cor = len(rec.loc[rec["Correctness"]==True])
    rec_total = len(rec)
    rec_percent = rec_cor/rec_total
    
    it_cor = len(it.loc[it["Correctness"]==True])
    it_total = len(it)
    it_percent = it_cor/it_total
    
    data = [
        Algorithm, Comprehension,
        rec_cor, rec_total, rec_percent, R_ResponseTimeMean, R_ResponseTimeDeviation,
        it_cor, it_total, it_percent, I_ResponseTimeMean, I_ResponseTimeDeviation,
    ]
    
    result_df = result_df.append(pd.DataFrame([data], columns=cols))
    
df_recursive = df_array_independent[0]     
df_recursive = df_recursive.append(df_array_independent[2])

df_iterative = df_array_independent[1]     
df_iterative = df_iterative.append(df_array_independent[3])

rec_cor = len(df_recursive.loc[df_recursive["Correctness"]==True])
rec_total = len(df_recursive)
rec_percent = rec_cor/rec_total

it_cor = len(df_iterative.loc[df_iterative["Correctness"]==True])
it_total = len(df_iterative)
it_percent = it_cor/it_total

result_df = result_df.append(pd.DataFrame(
    [["Total", "Total",
    rec_cor, rec_total, rec_percent, int(df_recursive["ResponseTime"].mean()), int(df_recursive["ResponseTime"].std()),
    it_cor, it_total, it_percent, int(df_iterative["ResponseTime"].mean()), int(df_iterative["ResponseTime"].std())]],
    columns=cols))

print("Write total means to excel sheet")
result_df.to_excel(writer, sheet_name="corrects.info")
writer.save()
result_df

Write total means to excel sheet


,Algorithm,Comprehension,rec correct,rec total Answers,rec percent,R_ResponseTimeMean,R_ResponseTimeDeviation,it correct,it total Answers,it percent,I_ResponseTimeMean,I_ResponseTimeDeviation
0,BinarySearch,Bottom-Up,7,7,1.0,441,201,3,3,1.0,283,261
0,BinarySearch,Top-Down,3,3,1.0,202,99,9,9,1.0,322,301
0,BubbleSort,Bottom-Up,13,13,1.0,260,156,8,8,1.0,261,218
0,BubbleSort,Top-Down,17,17,1.0,27,28,16,16,1.0,401,492
0,Factorial,Bottom-Up,24,24,1.0,132,251,27,27,1.0,139,105
0,Factorial,Top-Down,25,25,1.0,77,35,19,19,1.0,109,78
0,Fibonacci,Bottom-Up,13,13,1.0,180,143,18,18,1.0,240,144
0,Fibonacci,Top-Down,16,16,1.0,200,154,23,23,1.0,180,113
0,IntegerBinary,Bottom-Up,7,7,1.0,420,516,7,7,1.0,399,403
0,IntegerBinary,Top-Down,12,12,1.0,130,92,12,12,1.0,210,98


create means of means for total